In [ ]:
from jupyter_plotly_dash import JupyterDash

import base64
import dash 
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "1234"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = dash.Dash()
#app = JupyterDash('JupyterDash')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(
    href="https://www.snhu.edu",
    children=[
        html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())))
    ]),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard Jehu Domenech Ortiz'))),
    html.Hr(),
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
    html.Div(
        className='row',
        style={'display':'flex'},
        children=[
            html.Button(id='button-one', n_clicks=0, children='Water Rescue'),
            html.Button(id='button-two', n_clicks=0, children='Mountain or Wilderness Rescue'),
            html.Button(id='button-three', n_clicks=0, children='Disaster Rescue or Individual Tracking'),
            html.Button(id='button-four', n_clicks=0, children='Reset')
        ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################


    
@app.callback(Output('datatable-id','data'),
              [Input('button-one', 'n_clicks'),
               Input('button-two', 'n_clicks'),
               Input('button-three', 'n_clicks'),
               Input('button-four', 'n_clicks')])
def update_dashboard(bt1, bt2, bt3, bt4):
### FIX ME Add code to filter interactive data table with MongoDB queries
    trigerred_id = dash.callback_context.triggered[0]['prop_id'].split('.')[0]

    if("button-four" == trigerred_id):
        df = pd.DataFrame.from_records(shelter.read({}))
    elif("button-one" == trigerred_id):
        df = pd.DataFrame(list(shelter.read({'$and': [{'$or':[{'breed':'Labrador Retriever Mix'}, 
            {'breed':'Chesapeake Bay Retriever'},
            {'breed':'Newfoundland'}]}, {'sex_upon_outcome':'Intact Female'}, 
            {'age_upon_outcome_in_weeks':{'$gt':26, '$lt':156}}
                                                     ]})))
    elif("button-two" == trigerred_id):
        df = pd.DataFrame(list(shelter.read({'$and': [{'$or':[{'breed':'German Shepherd'}, 
            {'breed':'Alaskan Malamute'},
            {'breed':'Old English Sheepdog'}, {'breed':'Siberian Husky'}, {'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, 
            {'age_upon_outcome_in_weeks':{'$gt':26, '$lt':156}}
                                                     ]})))
    elif("button-three" == trigerred_id):
        df = pd.DataFrame(list(shelter.read({'$and': [{'$or':[{'breed':'Doberman Pinscher'}, 
            {'breed':'German Shepherd'},
            {'breed':'Golden Retriever'}, {'breed':'Bloodhound'}, {'breed':'Rottweiler'}]}, 
            {'sex_upon_outcome':'Intact Male'}, 
            {'age_upon_outcome_in_weeks':{'$gt':26, '$lt':300}}
                                                     ]})))
         
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
        
        
    return (data)
    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    df = pd.DataFrame.from_dict(viewData)

    fig = px.pie(df, values=df.get('breed').value_counts(),names=df.get('breed').unique())
    return[dcc.Graph(figure=fig)]
    

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])#\

#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.
def update_map(viewData):
    if viewData is None:
        return []
    else:
        dff = pd.DataFrame.from_dict(viewData)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.75,-97.48], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[1,9])
                    ])
                ])
            ])
    ]


app.run_server(reloader_type=False)
#app

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [19/Feb/2023 22:25:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:25] "GET /_favicon.ico?v=1.10.0 HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:25] "POST /_dash-update-component HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/anaconda/lib/python3.6/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/usr/local/anaconda/lib/python3.6/site-packages/dash/dash.py", line 1

127.0.0.1 - - [19/Feb/2023 22:25:25] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [19/Feb/2023 22:25:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:26] "POST /_dash-update-component HTTP/1.1" 200 -


1 age_upon_outcome animal_id animal_type  \
0   1          3 years   A746874         Cat   
1   3          2 years   A716330         Dog   
2   4         7 months   A733653         Cat   
3   2           1 year   A725717         Cat   
4   5          2 years   A691584         Dog   
5   6          5 years   A696004         Dog   
6   8           1 year   A736551         Dog   
7   7          2 years   A673830         Dog   
8  10         3 months   A664290         Cat   
9   9          3 years   A720214         Dog   

                                      breed         color date_of_birth  \
0                    Domestic Shorthair Mix   Black/White    2014-04-10   
1                   Chihuahua Shorthair Mix   Brown/White    2013-11-18   
2                               Siamese Mix    Seal Point    2016-01-25   
3                    Domestic Shorthair Mix  Silver Tabby    2015-05-02   
4                    Labrador Retriever Mix     Tan/White    2012-11-06   
5                  Cardigan Welsh Corgi Mix   Sable/White    2010-01-27   
6  Labrador Retriever/Australian Cattle Dog         Black    2015-10-12   
7                              Pit Bull Mix   Black/White    2012-03-03   
8                    Domestic Shorthair Mix        Tortie    2013-09-01   
9                    Labrador Retriever Mix     Red/White    2013-02-04   

              datetime            monthyear      name outcome_subtype  \
0  2017-04-11 09:00:00  2017-04-11T09:00:00                      SCRP   
1  2015-12-28 18:43:00  2015-12-28T18:43:00     Frank                   
2  2016-08-27 18:11:00  2016-08-27T18:11:00     Kitty                   
3  2016-05-06 10:49:00  2016-05-06T10:49:00                      SCRP   
4  2015-05-30 13:48:00  2015-05-30T13:48:00      Luke                   
5  2015-01-28 10:39:00  2015-01-28T10:39:00      Lucy     Rabies Risk   
6  2016-11-27 18:00:00  2016-11-27T18:00:00      *Mia                   
7  2014-03-19 15:15:00  2014-03-19T15:15:00     *Seth      Aggressive   
8  2013-12-08 14:58:00  2013-12-08T14:58:00   *Taylor                   
9  2016-02-11 12:41:00  2016-02-11T12:41:00  Blessing                   

      outcome_type sex_upon_outcome  location_lat  location_long  \
0         Transfer    Neutered Male     30.506658     -97.340878   
1         Adoption    Neutered Male     30.759575     -97.552375   
2         Adoption    Intact Female     30.318806     -97.724038   
3         Transfer    Spayed Female     30.652598     -97.741996   
4  Return to Owner    Neutered Male     30.710482     -97.562297   
5       Euthanasia    Spayed Female     30.673737     -97.707972   
6         Adoption    Spayed Female     30.444321     -97.732698   
7       Euthanasia    Neutered Male     30.295426     -97.313664   
8         Adoption    Spayed Female     30.758311     -97.618292   
9         Adoption    Spayed Female     30.387065     -97.368434   

   age_upon_outcome_in_weeks animal  
0                 156.767857   None  
1                 110.111409   None  
2                  30.822520   None  
3                  52.921528   None  
4                 133.653571   None  
5                 261.063393   None  
6                  58.964286   None  
7                 106.662202   None  
8                  14.089087   None  
9                 157.504067   None

...

127.0.0.1 - - [19/Feb/2023 22:25:31] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:32] "POST /_dash-update-component HTTP/1.1" 200 -


1 age_upon_outcome animal_id animal_type  \
0   1          3 years   A746874         Cat   
1   3          2 years   A716330         Dog   
2   4         7 months   A733653         Cat   
3   2           1 year   A725717         Cat   
4   5          2 years   A691584         Dog   
5   6          5 years   A696004         Dog   
6   8           1 year   A736551         Dog   
7   7          2 years   A673830         Dog   
8  10         3 months   A664290         Cat   
9   9          3 years   A720214         Dog   

                                      breed         color date_of_birth  \
0                    Domestic Shorthair Mix   Black/White    2014-04-10   
1                   Chihuahua Shorthair Mix   Brown/White    2013-11-18   
2                               Siamese Mix    Seal Point    2016-01-25   
3                    Domestic Shorthair Mix  Silver Tabby    2015-05-02   
4                    Labrador Retriever Mix     Tan/White    2012-11-06   
5                  Cardigan Welsh Corgi Mix   Sable/White    2010-01-27   
6  Labrador Retriever/Australian Cattle Dog         Black    2015-10-12   
7                              Pit Bull Mix   Black/White    2012-03-03   
8                    Domestic Shorthair Mix        Tortie    2013-09-01   
9                    Labrador Retriever Mix     Red/White    2013-02-04   

              datetime            monthyear      name outcome_subtype  \
0  2017-04-11 09:00:00  2017-04-11T09:00:00                      SCRP   
1  2015-12-28 18:43:00  2015-12-28T18:43:00     Frank                   
2  2016-08-27 18:11:00  2016-08-27T18:11:00     Kitty                   
3  2016-05-06 10:49:00  2016-05-06T10:49:00                      SCRP   
4  2015-05-30 13:48:00  2015-05-30T13:48:00      Luke                   
5  2015-01-28 10:39:00  2015-01-28T10:39:00      Lucy     Rabies Risk   
6  2016-11-27 18:00:00  2016-11-27T18:00:00      *Mia                   
7  2014-03-19 15:15:00  2014-03-19T15:15:00     *Seth      Aggressive   
8  2013-12-08 14:58:00  2013-12-08T14:58:00   *Taylor                   
9  2016-02-11 12:41:00  2016-02-11T12:41:00  Blessing                   

      outcome_type sex_upon_outcome  location_lat  location_long  \
0         Transfer    Neutered Male     30.506658     -97.340878   
1         Adoption    Neutered Male     30.759575     -97.552375   
2         Adoption    Intact Female     30.318806     -97.724038   
3         Transfer    Spayed Female     30.652598     -97.741996   
4  Return to Owner    Neutered Male     30.710482     -97.562297   
5       Euthanasia    Spayed Female     30.673737     -97.707972   
6         Adoption    Spayed Female     30.444321     -97.732698   
7       Euthanasia    Neutered Male     30.295426     -97.313664   
8         Adoption    Spayed Female     30.758311     -97.618292   
9         Adoption    Spayed Female     30.387065     -97.368434   

   age_upon_outcome_in_weeks animal  
0                 156.767857   None  
1                 110.111409   None  
2                  30.822520   None  
3                  52.921528   None  
4                 133.653571   None  
5                 261.063393   None  
6                  58.964286   None  
7                 106.662202   None  
8                  14.089087   None  
9                 157.504067   None

127.0.0.1 - - [19/Feb/2023 22:25:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [19/Feb/2023 22:25:33] "POST /_dash-update-component HTTP/1.1" 200 -


1 age_upon_outcome animal_id animal_type  \
0   1          3 years   A746874         Cat   
1   3          2 years   A716330         Dog   
2   4         7 months   A733653         Cat   
3   2           1 year   A725717         Cat   
4   5          2 years   A691584         Dog   
5   6          5 years   A696004         Dog   
6   8           1 year   A736551         Dog   
7   7          2 years   A673830         Dog   
8  10         3 months   A664290         Cat   
9   9          3 years   A720214         Dog   

                                      breed         color date_of_birth  \
0                    Domestic Shorthair Mix   Black/White    2014-04-10   
1                   Chihuahua Shorthair Mix   Brown/White    2013-11-18   
2                               Siamese Mix    Seal Point    2016-01-25   
3                    Domestic Shorthair Mix  Silver Tabby    2015-05-02   
4                    Labrador Retriever Mix     Tan/White    2012-11-06   
5                  Cardigan Welsh Corgi Mix   Sable/White    2010-01-27   
6  Labrador Retriever/Australian Cattle Dog         Black    2015-10-12   
7                              Pit Bull Mix   Black/White    2012-03-03   
8                    Domestic Shorthair Mix        Tortie    2013-09-01   
9                    Labrador Retriever Mix     Red/White    2013-02-04   

              datetime            monthyear      name outcome_subtype  \
0  2017-04-11 09:00:00  2017-04-11T09:00:00                      SCRP   
1  2015-12-28 18:43:00  2015-12-28T18:43:00     Frank                   
2  2016-08-27 18:11:00  2016-08-27T18:11:00     Kitty                   
3  2016-05-06 10:49:00  2016-05-06T10:49:00                      SCRP   
4  2015-05-30 13:48:00  2015-05-30T13:48:00      Luke                   
5  2015-01-28 10:39:00  2015-01-28T10:39:00      Lucy     Rabies Risk   
6  2016-11-27 18:00:00  2016-11-27T18:00:00      *Mia                   
7  2014-03-19 15:15:00  2014-03-19T15:15:00     *Seth      Aggressive   
8  2013-12-08 14:58:00  2013-12-08T14:58:00   *Taylor                   
9  2016-02-11 12:41:00  2016-02-11T12:41:00  Blessing                   

      outcome_type sex_upon_outcome  location_lat  location_long  \
0         Transfer    Neutered Male     30.506658     -97.340878   
1         Adoption    Neutered Male     30.759575     -97.552375   
2         Adoption    Intact Female     30.318806     -97.724038   
3         Transfer    Spayed Female     30.652598     -97.741996   
4  Return to Owner    Neutered Male     30.710482     -97.562297   
5       Euthanasia    Spayed Female     30.673737     -97.707972   
6         Adoption    Spayed Female     30.444321     -97.732698   
7       Euthanasia    Neutered Male     30.295426     -97.313664   
8         Adoption    Spayed Female     30.758311     -97.618292   
9         Adoption    Spayed Female     30.387065     -97.368434   

   age_upon_outcome_in_weeks animal  
0                 156.767857   None  
1                 110.111409   None  
2                  30.822520   None  
3                  52.921528   None  
4                 133.653571   None  
5                 261.063393   None  
6                  58.964286   None  
7                 106.662202   None  
8                  14.089087   None  
9                 157.504067   None

127.0.0.1 - - [19/Feb/2023 22:25:34] "POST /_dash-update-component HTTP/1.1" 200 -
